In [27]:
import cv2
import numpy as np

# โหลดภาพ
image_path = "bloodcell.jpg"
image = cv2.imread(image_path)
if image is None:
    print("ไม่สามารถโหลดภาพได้ กรุณาตรวจสอบที่อยู่ไฟล์")
    exit()

# แปลงเป็นภาพ grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# ปรับปรุงภาพด้วย CLAHE ที่มีค่า clipLimit สูงขึ้นเพื่อเพิ่ม contrast
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
enhanced = clahe.apply(gray)

# ใช้ Gaussian Blur เพื่อลดสัญญาณรบกวน
blurred = cv2.GaussianBlur(enhanced, (5, 5), 0)

# ทดลองใช้ Circle Hough Transform เพื่อตรวจจับวงกลม (เซลล์เม็ดเลือด)
circles = cv2.HoughCircles(
    blurred, 
    cv2.HOUGH_GRADIENT, 
    dp=1, 
    minDist=20, 
    param1=50, 
    param2=25,  # ลดค่านี้ลงเพื่อตรวจจับวงกลมที่ไม่ชัดเจน
    minRadius=8,  # ปรับตามขนาดของเซลล์ในภาพจริง
    maxRadius=25  # ปรับตามขนาดของเซลล์ในภาพจริง
)

output = image.copy()

# ถ้าตรวจพบวงกลม
if circles is not None:
    # แปลงค่าพารามิเตอร์เป็นจำนวนเต็ม
    circles = np.uint16(np.around(circles))
    
    # วาดวงกลมและใส่หมายเลขกำกับ
    for i, (x, y, r) in enumerate(circles[0, :]):
        # วาดวงกลมรอบเซลล์
        cv2.circle(output, (x, y), r, (0, 255, 0), 2)
        # วาดจุดศูนย์กลาง
        cv2.circle(output, (x, y), 2, (0, 0, 255), 3)
        # ใส่หมายเลขกำกับ
        cv2.putText(output, str(i+1), (x-10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    print(f"จำนวนเซลล์ที่ตรวจพบ: {len(circles[0])}")
else:
    print("ไม่พบเซลล์ใดๆ กรุณาปรับค่าพารามิเตอร์")

# แสดงภาพผลลัพธ์
cv2.imshow("Original", image)
cv2.imshow("Enhanced", enhanced)
cv2.imshow("Detected Cells", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ถ้ายังไม่พอใจกับผลลัพธ์ ลองใช้วิธีอื่นโดยใช้ watershed algorithm
# สร้างฟังก์ชันสำรองที่ใช้วิธี watershed
def detect_cells_watershed(img):
    # แปลงเป็นภาพ grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # ปรับปรุงภาพด้วย CLAHE
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    
    # Gaussian blur
    blurred = cv2.GaussianBlur(enhanced, (5, 5), 0)
    
    # ใช้ binary threshold + OTSU
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    
    # Noise removal with morphological operations
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)
    
    # Sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.25*dist_transform.max(), 255, 0)
    
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    # Marker labelling
    _, markers = cv2.connectedComponents(sure_fg)
    
    # Add one to all labels so that background is not 0, but 1
    markers = markers + 1
    
    # Mark the region of unknown with zero
    markers[unknown == 255] = 0
    
    # Apply watershed
    markers = cv2.watershed(img, markers)
    
    # Results
    result_img = img.copy()
    
    # Count the unique labels except background (1) and boundary markers (-1)
    unique_labels = np.unique(markers)
    cell_count = len(unique_labels) - 2  # -2 for removing background (1) and boundaries (-1)
    
    # Draw the boundaries and number the cells
    cell_id = 0
    for label in unique_labels:
        if label <= 1:  # Skip background (1) and boundaries (-1)
            continue
            
        cell_id += 1
        mask = np.zeros_like(markers, dtype=np.uint8)
        mask[markers == label] = 255
        
        # Find the contour of the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(contours) > 0:
            # Calculate centroid
            M = cv2.moments(contours[0])
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                
                # Draw contour and label
                cv2.drawContours(result_img, contours, -1, (0, 255, 0), 2)
                cv2.putText(result_img, str(cell_id), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    print(f"จำนวนเซลล์ที่ตรวจพบ (Watershed): {cell_id}")
    return result_img, binary, markers

# ใช้วิธี watershed และแสดงผล
watershed_output, binary, markers = detect_cells_watershed(image)
cv2.imshow("Watershed Result", watershed_output)
cv2.imshow("Binary", binary)
cv2.imshow("Markers", np.uint8(markers * 10))  # คูณด้วย 10 เพื่อให้เห็นความแตกต่างชัดเจน
cv2.waitKey(0)
cv2.destroyAllWindows()

จำนวนเซลล์ที่ตรวจพบ: 61
จำนวนเซลล์ที่ตรวจพบ (Watershed): 88


In [59]:
import cv2
import numpy as np

# โหลดภาพ
image_path = "bloodcell.jpg"
image = cv2.imread(image_path)
if image is None:
    print("ไม่สามารถโหลดภาพได้ กรุณาตรวจสอบที่อยู่ไฟล์")
    exit()

# แปลงเป็นภาพ grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# ปรับความสว่างและความคมชัด
alpha = 1.5  # ปรับความคมชัด
beta = 10    # ปรับความสว่าง
adjusted = cv2.convertScaleAbs(gray, alpha=alpha, beta=beta)

# ใช้ CLAHE เพื่อปรับปรุง contrast ท้องถิ่น
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
enhanced = clahe.apply(adjusted)

# ใช้ Gaussian Blur เพื่อลดสัญญาณรบกวน
blurred = cv2.GaussianBlur(enhanced, (5, 5), 0)

# ทำ threshold เพื่อเน้นเฉพาะจุดสีเข้มตรงกลางเซลล์
# ปรับค่า threshold ตามความเข้มของจุดสีเข้มในภาพของคุณ
_, binary = cv2.threshold(blurred, 70, 255, cv2.THRESH_BINARY_INV)

# ใช้ morphological operations เพื่อกำจัดสัญญาณรบกวนเล็กๆ
kernel = np.ones((1, 1), np.uint8)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)

# หา contours ของจุดสีเข้ม
contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# กรองเอาเฉพาะ contours ที่มีขนาดเหมาะสม
min_area = 1/20  # ขนาดเล็กสุดของจุดที่ถือว่าเป็นเซลล์
max_area = 100  # ขนาดใหญ่สุดของจุดที่ถือว่าเป็นเซลล์
filtered_contours = []

for contour in contours:
    area = cv2.contourArea(contour)
    if min_area < area < max_area:
        filtered_contours.append(contour)

# หาจุดศูนย์กลางของแต่ละ contour
centroids = []
for contour in filtered_contours:
    M = cv2.moments(contour)
    if M["m00"] > 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        centroids.append((cX, cY))

# วาดวงกลมรอบเซลล์และเพิ่มเลขกำกับ
# result = image.copy()
# for i, (cX, cY) in enumerate(centroids):
#     # วาดวงกลมรอบเซลล์
#     cv2.circle(result, (cX, cY), 15, (0, 255, 0), 2)
#     # ใส่หมายเลขกำกับ
#     cv2.putText(result, str(i+1), (cX-10, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# แสดงจำนวนเซลล์ที่ตรวจพบ
print(f"จำนวนเซลล์ที่ตรวจพบ: {len(centroids)}")

# แสดงภาพและบันทึกผลลัพธ์
cv2.imshow("Original", image)
cv2.imshow("Enhanced", enhanced)
cv2.imshow("Binary", opening)
cv2.imshow("Detected Cells", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# บันทึกผลลัพธ์
cv2.imwrite("detected_cells_result.jpg", result)

จำนวนเซลล์ที่ตรวจพบ: 127


True